<a href="https://colab.research.google.com/github/FredyAlexanderOrjuela2015/Clase-de-Sistemas-Avnzados-de-Producci-n/blob/main/Clase_2_Taller_1_Datos_Con_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Clase 2 (2h): Estadística descriptiva y visualización (univariado y bivariado)

**Objetivos**
- Describir variables numéricas y categóricas.
- Detectar atípicos (regla 1.5·IQR).
- Visualizar con histogramas, cajas y barras.
- Explorar relaciones bivariadas: numérico↔numérico y categórico↔categórico.


In [ ]:

import sys, platform, math
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
%matplotlib inline
print("Python:", sys.version.split()[0], "|", platform.platform())


In [ ]:

# Dataset base (si no existe 'da')
if 'da' not in globals():
    np.random.seed(42)
    n = 800
    da = pd.DataFrame({
        "SEQN": np.arange(1, n+1),
        "RIAGENDR": np.random.choice([1, 2], size=n),
        "RIDAGEYR": np.random.randint(18, 80, size=n),
        "BMXWT": np.round(np.random.normal(71, 13, size=n), 1),
        "BMXHT": np.round(np.random.normal(168, 9, size=n), 1),
    })
    da["BMXBMI"] = np.round(da["BMXWT"] / (da["BMXHT"]/100)**2, 1)
    da.loc[np.random.choice(da.index, size=20, replace=False), "BMXWT"] = np.nan

# Recodificar
if "RIAGENDR" in da.columns:
    da["sexo"] = da["RIAGENDR"].map({1:"Masculino", 2:"Femenino"})
if "BMXBMI" in da.columns:
    def cat_imc(x):
        if pd.isnull(x): return np.nan
        if x < 18.5: return "Bajo peso"
        if x < 25:   return "Normal"
        if x < 30:   return "Sobrepeso"
        return "Obesidad"
    da["IMC_cat"] = da["BMXBMI"].apply(cat_imc)
da.head()


## Descriptivos univariados

In [ ]:

num_cols = da.select_dtypes(include=[np.number]).columns
resumen = da[num_cols].describe(percentiles=[0.25, 0.5, 0.75]).T
resumen["IQR"] = resumen["75%"] - resumen["25%"]
resumen


In [ ]:

# Categóricas: frecuencias y proporciones
cat_cols = [c for c in da.columns if da[c].dtype == "O"]
tablas = {}
for c in cat_cols:
    vc = da[c].value_counts(dropna=False)
    prop = (vc / len(da)).round(3)
    tablas[c] = pd.DataFrame({"frecuencia": vc, "proporcion": prop})
tablas.get("IMC_cat", None)


## Atípicos con 1.5·IQR y visualizaciones

In [ ]:

def iqr_outliers(s):
    s = s.dropna()
    q1, q3 = np.percentile(s, [25, 75])
    iqr = q3 - q1
    low, high = q1 - 1.5*iqr, q3 + 1.5*iqr
    return (low, high, (s < low).sum(), (s > high).sum())

target = "BMXBMI" if "BMXBMI" in da.columns else num_cols[0]
low, high, nlow, nhigh = iqr_outliers(da[target])
print(f"{target} → límites [{low:.2f}, {high:.2f}] | atípicos por abajo: {nlow}, por arriba: {nhigh}")

plt.figure(); da[target].dropna().hist(bins=25)
plt.title(f"Histograma de {target}"); plt.xlabel(target); plt.ylabel("Frecuencia"); plt.show()

plt.figure(); plt.boxplot(da[target].dropna(), vert=True)
plt.title(f"Diagrama de caja de {target}"); plt.ylabel(target); plt.show()


## Relaciones bivariadas

In [ ]:

# Numérico vs numérico
if {"RIDAGEYR","BMXBMI"}.issubset(da.columns):
    x, y = da["RIDAGEYR"], da["BMXBMI"]
    r = np.corrcoef(x.dropna(), y.dropna())[0,1]
    plt.figure(); plt.scatter(x, y, s=12)
    plt.title(f"Dispersión: Edad vs IMC (r≈{r:.2f})"); plt.xlabel("Edad (años)"); plt.ylabel("IMC"); plt.show()
else:
    print("No se encuentran 'RIDAGEYR' y 'BMXBMI'.")


In [ ]:

    # Categórico vs categórico
    import pandas as pd, matplotlib.pyplot as plt
    if {"sexo","IMC_cat"}.issubset(da.columns):
        ct = pd.crosstab(da["sexo"], da["IMC_cat"], dropna=False)
        print("Tabla de contingencia:
", ct, "
")
        (ct.T / ct.T.sum()).T.plot(kind="bar", stacked=True, rot=0)
        plt.title("Distribución de IMC por sexo (proporciones)")
        plt.xlabel("Sexo"); plt.ylabel("Proporción"); plt.legend(title="IMC_cat", bbox_to_anchor=(1.05,1), loc="upper left")
        plt.show()
    else:
        print("No se encuentran 'sexo' e 'IMC_cat'.")


## Q-Q Plot (opcional)

In [ ]:

import statsmodels.api as sm, matplotlib.pyplot as plt
var = "BMXBMI" if "BMXBMI" in da.columns else num_cols[0]
plt.figure(); sm.ProbPlot(da[var].dropna()).qqplot(line="s")
plt.title(f"Q-Q plot de {var}"); plt.show()



## Ejercicios (para tu repositorio)
1) Resumen de una variable numérica y detección de atípicos.  
2) Histograma y boxplot con comentario sobre la forma.  
3) Tabla `sexo × IMC_cat` con proporciones.  
4) Dispersograma de dos numéricas e interpretación de la correlación.



### Respuestas del estudiante (Markdown)
<div class="response-box">
Escribe aquí tu análisis y conclusiones para los puntos 1–4.
</div>


In [ ]:
# Respuestas del estudiante (Código)
# TODO: Añade aquí tu código para los ejercicios de la Clase 2